In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import OneHotEncoder

# Leitura e Criação de Dataframe

Aqui a gente recebe o dataset e tranforma ele em um dataframe

In [2]:
df_unistudents = pd.read_csv('../data/StudentPerformanceFactors.csv', low_memory=False)
pd.set_option('display.max_columns', None)

## Pre-processamento dos dados

Substituição dos valores nulos pela média e pela moda de suas respectivas colunas:

In [ ]:
from sklearn.impute import SimpleImputer

# Numericas
num_imputer = SimpleImputer(strategy='mean')
num_col = df_unistudents.select_dtypes(include=['float64', 'int64']).columns
df_unistudents[num_col] = num_imputer.fit_transform(df_unistudents[num_col])
df_unistudents[num_col] = df_unistudents[num_col].apply(np.round)

# Categoricas
cat_imputer = SimpleImputer(strategy='most_frequent')
cat_col = df_unistudents.select_dtypes(include=['object']).columns
df_unistudents[cat_col] = cat_imputer.fit_transform(df_unistudents[cat_col])

Vamos criar uma nova coluna chamada ChangesChange usando a coluna "Previous_Scores" (notas anteriores) e a 'Exam_Score' (notas atuais), com a finalidade de utilizá-la como variável alvo no algoritmo de classificação que desenvolveremos a seguir (apageremos as colunas "Previous_Scores" e "Exam_Score" em seguida para não dar conflito no algoritmo).

In [ ]:
def mudanca_de_nota(row):
    if row['Exam_Score'] > row['Previous_Scores']:
        return 'Increase'
    elif row['Exam_Score'] < row['Previous_Scores']:
        return 'Decrease'
    else:
        return 'No Change'

df_unistudents['Change_Grades'] = df_unistudents.apply(mudanca_de_nota, axis=1)

df_unistudents = df_unistudents.drop(['Previous_Scores', 'Exam_Score'], axis=1)

Aplicaremos o "One Hot Encoder" para converter as variáveis categóricas de uma maneira que aumente o desempenho dos modelos de machine learning. Aqui, transformaremos cada coluna categórica em uma nova coluna binária para indicar a presença da categoria ou a ausência.

Aplicação do hot_encoder (com exceção da coluna Change_Grades):

In [28]:
cat_col = df_unistudents.select_dtypes(include=['object']).columns
cat_col = cat_col.drop('Change_Grades')

one_hot_encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_columns = one_hot_encoder.fit_transform(df_unistudents[cat_col])

Criação de um DataFrame com as colunas codificadas:

In [29]:
encoded_df = pd.DataFrame(encoded_columns, columns=one_hot_encoder.get_feature_names_out(cat_col))

Concatenação do dataframe codificado com o original:

In [30]:
df_unistudents = pd.concat([df_unistudents, encoded_df], axis=1)

Remoção das colunas categóricas originais:

In [31]:
df_unistudents.drop(columns=cat_col, inplace=True)

In [ ]:
df_unistudents.head(35)

Repetiremos a matriz de correlação para verificar as mudanças (sem a Change_Grades que é string):

In [ ]:
cols = df_unistudents.columns
cols = cols.drop('Change_Grades')

correlation_matrix = df_unistudents[cols].corr()

plt.figure(figsize=(50, 45))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlação - Variáveis Numéricas")
plt.show()

Vamos transformar o tipo de arquivo para .parquet com o intuito de tornar o processo mais rápido

In [36]:
df_unistudents.to_parquet("new_unistudents.parquet", index=False)